In [1]:
import cv2 as cv
import numpy as np

In [2]:
# Load Image
img = cv.imread(filename='../data/images/zebra.png', flags=cv.IMREAD_GRAYSCALE)
cv.imshow(mat = img, winname = 'Original')
cv.waitKey(0)

-1

Se puede hacer uso de la primera derivada para el mejoramiento de la imagen. En este caso tomamos el vector gradiente:
$$ \nabla f = \begin{bmatrix} Gx \\ Gy \end{bmatrix} $$
Donde su modulo resulta ser:
$$ |\nabla f| = (Gx^2+Gy^2)^{1/2} $$
Y una aproximacion a esta es:
$$ |\nabla f| \approx |Gx|+|Gy| $$
Donde tenemos:
$$ Gx = \frac{\delta f}{\delta x} = f(x+1) - f(x) $$
$$ Gy = \frac{\delta f}{\delta y} = f(y+1) - f(y) $$
El operador Sobel usa dos kernels (generalmente de $ 3 \cdot 3 $) los cuales son convolucionados con la imagen general para calcular aproximaciones de las derivadas. 
Siendo un kernel de la imagen:
$$ A = \begin{bmatrix} z1 & z2 & z3 \\ z4 & z5 & z6 \\ z7 & z8 & z9 \end{bmatrix} $$
Y siendo la aproximacion del modulo del gradiente la siguiente:
$$ |\nabla f| \approx |(z2+2\cdot z8+z9)-(z1+2\cdot z2+z3)| + |(z3+2\cdot z6+z9)-(z1+2\cdot z4+z7)|$$
Entonces calculamos $ Gx $ y $ Gy $ de la siguiente manera:
$$ G_{x} = \begin{bmatrix} -1 & 0 & +1 \\ -2 & 0 & +2 \\ -1 & 0 & +1 \end{bmatrix} * I $$
$$ G_{y} = \begin{bmatrix} -1 & -2 & -1 \\ 0 & 0 & 0 \\ +1 & +2 & +1 \end{bmatrix} * I $$
El 2 en las matrices se usa para dar mayor importancia al pixel central.
Esto se generaliza para kernels de tamaño 1, 5 y 7.


In [3]:
kernels = [1,3,5,7]
for ksize in kernels:
    # Gradient X
    Gx = cv.Sobel(src = img, ddepth=cv.CV_16S, dx=1, dy=0, ksize=ksize)
    # Gradient Y
    Gy = cv.Sobel(src = img, ddepth=cv.CV_16S, dx=0, dy=1, ksize=ksize)
    
    # Convert image back to greyscale
    Gx = cv.convertScaleAbs(Gx, alpha=255/Gx.max())
    Gy = cv.convertScaleAbs(Gy, alpha=255/Gy.max())
    
    # Display
    display = np.hstack((Gx, Gy))
    cv.imshow(mat = display, winname = 'Sobel - Gx vs Gy - Kernel Size: '+str(ksize))
    cv.waitKey(0)

Se puede observar en las imagenes anteriores el efecto del tamaño del kernel en la imagen.
En las mismas se puede apreciar que cuando se aplica la derviada en $ x $ se realzan las lineas verticales ya que hay una mayor derivada cuando hay cambios de color abruptos en el eje $ x $ y cuando se aplica la derivada en $ y $ se realzan las lineas horizontales ya que es anologo al caso anterior.
Se puede obtener mejores aproximaciones a la gradiente cuando el kernel es 3 con el uso del operador Scharr, ya que Sobel puede tener resultados imprecisos dado que es una aproximacion. Scharr es mas precisa que Sobel, y utiliza los siguientes kernels:
$$ Gx_{mat} = \begin{bmatrix} -3 & 0 & +3 \\ -10 & 0 & +10 \\ -3 & 0 & +3 \end{bmatrix} $$
$$ Gy_{mat} = \begin{bmatrix} -3 & -10 & -3 \\ 0 & 0 & 0 \\ +3 & +10 & +3 \end{bmatrix} $$
Y se calcula el gradiente de la siguiente manera:
$$ G_{x} = Gx_{mat} * I $$
$$ G_{y} = Gy_{mat} * I $$


In [4]:
# Gradient X
Gx_scharr = cv.Scharr(src = img, ddepth=cv.CV_16S, dx=1, dy=0)
# Gradient Y
Gy_scharr = cv.Scharr(src = img, ddepth=cv.CV_16S, dx=0, dy=1)

# Convert image back to greyscale
Gx_scharr = cv.convertScaleAbs(Gx_scharr, alpha=255/Gx_scharr.max())
Gy_scharr = cv.convertScaleAbs(Gy_scharr, alpha=255/Gy_scharr.max())

# Display
display_scharr = np.hstack((Gx_scharr, Gy_scharr))
cv.imshow(mat = display_scharr, winname = 'Scharr - Gx vs Gy - Kernel Size: '+str(3))
cv.waitKey(0)

-1

En este caso no hay diferencia apreciable entre el gradiente de Sobel y el de Scharr

In [5]:
# Gradient X
Gx = cv.Sobel(src = img, ddepth=cv.CV_16S, dx=1, dy=0, ksize=3)
# Gradient Y
Gy = cv.Sobel(src = img, ddepth=cv.CV_16S, dx=0, dy=1, ksize=3)
    
# Convert image back to greyscale
Gx = cv.convertScaleAbs(Gx, alpha=255/Gx.max())
Gy = cv.convertScaleAbs(Gy, alpha=255/Gy.max())
grad = cv.addWeighted(Gx, 0.5, Gy, 0.5, 0)
display_grad = np.hstack((img, grad))
cv.imshow(mat = display_grad, winname = 'Original vs Gradient')
cv.waitKey(0)

-1

Finalmente ponderamos ambas imagenes $ Gx $ y $ Gy $ de los gradientes de Sobel de kernel $ 3 \cdot 3 $ y conseguimos una imagen que representa la "derivada" de la imagen original. Donde se ve en colores claros donde hay cambios aprubtos de color.